In [30]:
import pandas as pd
from pydataxm import *
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer, Date, Float  # Importar tipos necesarios
import  os
import psycopg2

In [15]:
host = os.environ.get("POSTGRES_HOST")
port = int(os.environ.get("POSTGRES_PORT"))
database = os.environ.get("POSTGRES_DB")
user = os.environ.get("POSTGRES_USER")
password = os.environ.get("POSTGRES_PASSWORD")

# Crear el engine de SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [16]:
#Definir periodo de los datos
today = dt.date.today()
month_ago = today - dt.timedelta(days=360) # Approximation of a month

In [17]:
#Conexion a la API
objetoAPI = pydataxm.ReadDB()

In [41]:
#Descargar los datos
df_DemandaReal = objetoAPI.request_data(
                        "DemaReal",           #Se indica el nombre de la métrica tal como se llama en el campo metricId
                        "Agente",
                        month_ago,#Se indica el nombre de la entidad tal como se llama en el campo Entity
                        today     #Corresponde a la fecha inicial de la consulta
                        )

#Renombrar
df_DemandaReal.rename(columns={'Values_code': 'pkAgente',
                               'Id':'IdDemandaReal',
                              }, inplace=True)

value_hour_cols = [col for col in df_DemandaReal.columns if col.startswith('Values_Hour')]

df_DemandaReal = pd.melt(df_DemandaReal,
                   id_vars=[col for col in df_DemandaReal.columns if not col.startswith('Values_Hour')],
                   value_vars=value_hour_cols,
                   var_name='hora',
                   value_name='demanda_real')

# Clean up the 'hora' column to extract just the hour number
df_DemandaReal['hora'] = df_DemandaReal['hora'].str.replace('Values_Hour', '').astype(int)


df_DemandaReal["Date"] = pd.to_datetime(df_DemandaReal["Date"]).dt.date  # Convertir a date (sin hora)
df_DemandaReal["fecha_actualizacion"] = today


In [42]:
df_DemandaReal.head()

,IdDemandaReal,pkAgente,Date,hora,demanda_real,fecha_actualizacion
0,Agente,AAGG,2024-06-06,1,122.17,2025-06-01
1,Agente,ADCG,2024-06-06,1,NaN,2025-06-01
2,Agente,ASCC,2024-06-06,1,2865.57,2025-06-01
3,Agente,BCCG,2024-06-06,1,17.00,2025-06-01
4,Agente,BIAC,2024-06-06,1,27431.95,2025-06-01


In [44]:
# Insertar datos en PostgreSQL (modo 'append' para no borrar datos existentes)
with engine.begin() as conn:  # <- Se hace COMMIT automático al final
    df_DemandaReal.to_sql(
    name='demanda_real',          # Nombre de la tabla (sin esquema aquí)
    con=engine,
    schema='tablasxm_dev',       # Esquema especificado aquí
    if_exists='replace',          # Opción: 'append', 'replace' o 'fail'
    index=True,                 # Incluir el índice del DataFrame
    dtype={
          'iddemandaReal': String(),      # TEXT en PostgreSQL (equivale a String de SQLAlchemy)
        'pkagente': String(),           # INT4 en PostgreSQL
        'date': Date(),                  # DATE
        'hora': Integer(),               # INT4
        'demanda_real': Float() ,         # FLOAT4 en PostgreSQL
        'fecha_actualizacion': Date()
    })




print("Datos insertados correctamente.")

Datos insertados correctamente.


In [45]:
print(len(df_DemandaReal))

1055928
